# Open MODIS (hdf4)  Data in Python 

The **MOD**erate-resolution **I**maging **S**pectroradiometer is a sensor that 
produces multispectral data with 500m spatial resolution and 1-2 day temporal 
resolution. MODIS data is the basis for some common data products
including:
  * Burned Area
  * Sea Surface Temperature
  * Aerosol
  * Snow
  * Sea Ice
  
All of these products are available in the hdf4 format.

  
## Learning objectives
  * Introduce hierarchical data formats
  * Import hdf4 MODIS data using rioxarray
  * Query hdf4 metadata to learn about its structure

## Why use hierarchical data formats?

Hierarchical data files are:
  * **Self-Describing**
  * **Store Heterogeneous Data**
  * **Support Data Slicing**
  * **An Open Format**
  
...and of course we want to use HDFs because many organizations recognize 
the advantages and have made useful data available in this format.

### Hierarchical data formats can contain multiple types of data in one file

Think about the files and folders stored on your computer. You might have 
a data directory with some temperature data for multiple field sites. This 
temperature data is collected every minute and summarized on an hourly, 
daily and weekly basis.

Within ONE HDF file, you can store a similar set of data organized in the same 
way that you might organize files and folders on your computer.

![Hierarchical file structure can contain many types of data in one file](https://www.earthdatascience.org/images/earth-analytics/hierarchical-data-formats/hdf5-example-data-structure-with-metadata.jpg)

### hdf4 files can be opened in Python using rioxarray

Similar to opening **.tif** files using rasterio, 
you will use a context manager to open hdf4 files. Because the data are 
hierarchical, you will have to loop through the main dataset and the 
subdatasets nested within the main dataset to access the reflectance 
data (the bands) and the qa layers. 

In [ ]:
import os

import earthpy as et
import earthpy.plot as ep
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy.ma as ma
from rasterio._base import get_dataset_driver
from rasterio.plot import plotting_extent
import rioxarray as rxr
from shapely.geometry import box

# Get the MODIS data
et.data.get_data('cold-springs-modis-h4')
# Get the fire boundary
et.data.get_data('cold-springs-fire')
os.chdir(os.path.join(et.io.HOME, 'earth-analytics', 'data'))

# This will make rioxarray run faster
rioxarray_option = rxr.set_options(export_grid_mapping=False)

In [ ]:
# Create a path to the pre-fire MODIS h4 data
modis_pre_path = os.path.join("cold-springs-modis-h4",
                              "07_july_2016",
                              "MOD09GA.A2016189.h09v05.006.2016191073856.hdf")
modis_pre_path

In [ ]:
# This returns a list of two objects
modis = rxr.open_rasterio(modis_pre_path,
                          masked=True)


modis

By adding two output variables, open_rasterio will save
the reflectance / band data in a separate object from
the qa (quality assurance) data.

In [ ]:
# By adding two output variables,
modis_qa, modis_data = rxr.open_rasterio(modis_pre_path,
                                         masked=True)

The fist object returned contains all of the qa (quality assurance) 
and metadata layers.


In [ ]:
# The first object is a qa object
modis_qa

In [ ]:
# The second contains your spectral data - ie the "bands"
modis_data

### Hierarchical data formats are self-describing

#### You can view metadata before opening the file in Python
The command below runs `gdalinfo` which allows you to view the metadata.
Calling a command using  `!` in  notebooks runs a bash command!

NOTE - this will ONLY work if `gdal` is installed and  
recognized properly on your computer.

In [ ]:
# View file metadata using bash
!gdalinfo cold-springs-modis-h4/07_july_2016/MOD09GA.A2016189.h09v05.006.2016191073856.hdf

#### View metadata in Python

In [ ]:
modis_data.attrs

In [ ]:
modis_data.attrs['LONGNAME']

#### View CRS and Resolution

In [ ]:
# View entire metadata object for a MODIS band
modis_data.rio.crs

In [ ]:
# View data resolution
modis_data.rio.resolution()

#### View variable names
The variables with this pattern in the name:

`sur_refl_b01_1`  

are the bands which contain surface reflectance data. 

* **sur_refl_b01_1:** MODIS Band One
* **sur_refl_b02_1:** MODIS Band Two

In [ ]:
#  View variables in object
modis_data.rio.vars

##  Use Variables and Groups to Open Subsets of Your Data

To access the spatial information stored within your H4 file, you will need 
to loop through the subdatasets. Below you open a connection to the main h4 file, 
then you loop through each subdataset in the file. 

Below you loop through and print the name of each subdataset in the file.
Notice that there are some other layers in the file as well including the 
`state_1km` layer which contains the QA (cloud and quality assurance) information.

### Use  the  Group  Parameter  to Grab One Of the Sub Groups

You can use groups to  grab an entire subgroup rather than just 
specific layers or variables.
Select the "MODIS_Grid_500m_2D" group to get all the reflectance bands:

In [ ]:
# Notice that here, you get a single xarray object with just the bands that
# you want to work with
(rxr.open_rasterio(
        modis_pre_path,
        masked=True,
        )
    .squeeze())

### Select By  Variables

In [ ]:
# Open just the bands that you want to process
desired_bands = ["sur_refl_b01_1",
                 "sur_refl_b02_1",
                 "sur_refl_b03_1",
                 "sur_refl_b04_1",
                 "sur_refl_b07_1"]
# Notice that here, you get a single xarray object with just the bands that
# you want to work with
modis_bands = (
    rxr.open_rasterio(
        modis_pre_path,
        masked=True,
        )
    .squeeze())
modis_bands

##  Clip The Data

There are several different ways to clip your data. While for this assignment,
I suggest  that  you  use the same approach used in `open_clean_bands()` 
shown last week for class. If you are interested below there  is more  
information about a few different approaches.

In [ ]:
# Open the clip extent
fire_boundary_path = os.path.join("cold-springs-fire",
                                  "vector_layers",
                                  "fire-boundary-geomac",
                                  "co_cold_springs_20160711_2200_dd83.shp")
fire_boundary = gpd.read_file(fire_boundary_path)

Note that the data are in different CRS:

In [ ]:
print("The fire boundary crs is", fire_boundary.crs)
print("The MODIS crs is", modis_bands.rio.crs)

In [ ]:
# Get a reprojected bounding box
fire_reproj = 
# Clip the data
modis_clip = 

modis_clip

####  Speed Tests For Clipping Data

For those of you that want to see we tested two approaches  here and  
did find that repojecting using the `crs=` parameter is much slower  
and more memory intensive than the approach above.

For your homework either approach is ok!

In [ ]:
%%timeit
crop_bound_box = [box(*fire_boundary.total_bounds)]

# Reproject using the crs= parameter
modis_clip = (
    rxr.open_rasterio(
        modis_pre_path,
        masked=True,
        variable=desired_bands)
    .rio.clip(
        crop_bound_box,
        crs=fire_boundary.crs,
        from_disk=True))

In [ ]:
%%timeit
# Reproject the boundary
fire_reproj = fire_boundary.to_crs(modis_bands.rio.crs).total_bounds

modis_bands_not_clipped = rxr.open_rasterio(
    modis_pre_path,
    masked=True,
    variable=desired_bands)

modis_clip = modis_bands.rio.clip_box(*fire_reproj).squeeze()

##  Plot MODIS data

For  RGB or CIR  plotting,  you may want to select only the bands
needed for plotting

In [ ]:
# Select bands 1 and 2 and turn into array
rgb_bands = ['sur_refl_b01_1',
             'sur_refl_b03_1',
             'sur_refl_b04_1']
modis_rgb_xr = modis_clip[rgb_bands].to_array()
modis_rgb_xr

### Plot All MODIS Bands with EarthPy

You are now ready to plot your data using earthpy. Notice below that the 
images look washed out and there are large negative values in the data. 

This might be a good time to consider cleaning up your data by addressing 
`nodata` values.

In [ ]:
ep.plot_bands(modis_rgb_xr.values,
              scale=False,
              figsize=(10, 5),
              title=rgb_bands)
plt.show()

### RGB Image of MODIS Data Using EarthPy

Once you have your data cleaned up, you can plot an RGB image of your data 
to ensure that it looks correct!

[This lesson will help you remember what bands to use.](https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/intro-multispectral-data/)

In [ ]:
def clean_array_plot(xr_obj):
    """
    Take an  xarray object and replace null values with a mask for plotting

    Parameters
    ----------
    xr_obj : xarray object

    Returns
    -------
    A masked numpy array 

    """

    return 

In [ ]:
modis_rgb_clean = clean_array_plot(modis_rgb_xr)

# Plot MODIS RGB image -Note that this looks weird only because
# the  data  are clipped to such  a "small" extent
ep.plot_rgb(modis_rgb_clean,
            rgb=[0, 1, 2],
            title='RGB Image of MODIS Data',
            figsize=(7, 7))

plt.show()

In [ ]:
modis_rgb_unclipped = modis_bands_not_clipped[rgb_bands].to_array().squeeze()
modis_rgb_unclipped`

In [ ]:
# Plot unclipped data - RGB (just a demo)
modis_rgb_clean_not_clipped = clean_array_plot(modis_rgb_unclipped)

# Plot MODIS RGB image -Note that this looks weird only because
# the data are clipped to such a "small" extent
ep.plot_rgb(modis_rgb_clean_not_clipped,
            rgb=[0, 1, 2],
            title='RGB Image of MODIS Data',
            figsize=(7, 7))

plt.show()

##  Function  To  Open  and Clean  MODIS Data
Above we walked through the specific workflow for using MODIS data.
You can however use the same function that you used last week 
to process Landsat and NAIP data for MODIS.

The difference is now you  an  add the v ariable  parameter
to include the list of bands that you wish to grab from the
data.

In [ ]:
# Function in your homework assignment
def open_clean_bands(band_path,
                     crop_bound,
                     valid_range=None,
                     variable=None):
    """Open and clean a single landsat band.

    Parameters
    -----------
    band_path : string 
        A path to the array to be opened.
    crop_bound : geopandas GeoDataFrame
        A geopandas dataframe to be used to crop the raster data using rioxarray clip().
    valid_range:tuple (optional)
        A tuple of min and max range of values for the data. Default = None.
    variable : List
        A list of variables to be opened from the raster.

    Returns
    -----------
    band : xarray DataArray
        Cropped xarray DataArray

    """

    crop_bound_box = [box(*crop_bound.bounds.loc[0])]

    driver = get_dataset_driver(band_path)

    if :
        raise ValueError("OOPS I've encountered an HDF file  "
                         "which may have subgroups, please be sure  "
                         "to specify the variable=parameter so "
                         "I can clip the data properly.")

    try:
        band = (rxr.open_rasterio(
                    band_path,
                    masked=True,
                    variable=variable)
                .rio.clip(
                    crop_bound_box,
                    crs=crop_bound.crs,
                    all_touched=True,
                    from_disk=True)
                .squeeze())
    except:
        raise ValueError("Oops - I couldn't clip your data. "
                         "This may be due to a crs error.")

    # Mask the data to the valid range if a valid range tuple is provided
    if valid_range is not None:
        mask = ((band < valid_range[0]) | (band > valid_range[1]))
        band = band.where(~xr.where(mask, True, False))

    return band

In [ ]:
# # If you dont specify variables, this function will fail to clip the data
try:
    open_clean_bands(modis_pre_path,
                     crop_bound=fire_boundary)
except Exception as exception:
    print(exception)

Below, we use the bands object to subset the data. 

In [ ]:
cleaned_MODIS_data = open_clean_bands(modis_pre_path,
                                      crop_bound=fire_boundary,
                                      variable=desired_bands)
cleaned_MODIS_data

Note - it's ok if the plot below looks funky.
It represents very few pixels and pre-fire data does
not have a lot of variation in the pixels.

In [ ]:
# Plot unclipped data - RGB
modis_rgb = cleaned_MODIS_data[rgb_bands].to_array().squeeze()
modis_rgb_plot = clean_array_plot(modis_rgb)

# Plot MODIS RGB image - Note that this looks pixellated
# The data are clipped to a very "small" extent relative to the resolution
ep.plot_rgb(modis_rgb_plot,
            rgb=[0, 1, 2],
            title='RGB Image of MODIS Data',
            figsize=(7, 7))

plt.show()

In [ ]:
ep.plot_bands(modis_rgb_plot)
plt.show()